In [1]:
import requests 
from bs4 import BeautifulSoup
import csv 

url = "https://economictimes.indiatimes.com/tata-global-beverages-ltd/stocksupdate/companyid-12929.cms"
request = requests.get(url) 
  
soup_page = BeautifulSoup(request.content, 'html5lib') 
  
quotes = []
  
search_div = soup_page.find('div', attrs = {'class':'all'}) 
search_class = search_div.findAll('div', attrs = {'class':'eachStory'})  
for chunk in search_class: 
    quote = {} 
    text_heading = chunk.find('div', attrs = {'class':'headingText'})
    text_date = chunk.find('div', attrs = {'class':'storyDate'})
    quote['date'] = text_date.time.text
    quote['text'] = text_heading.h3.text + '. ' + chunk.p.text
    quotes.append(quote)
    
quotes

[{'date': '18 Nov,2019, 08:21AM IST',
  'text': 'Tata Global Beverages  Disclosures under Reg. 29(2) of SEBI (SAST) Regulations, 2011. The Exchange has received the disclosure under Regulation 29(2) of SEBI (Substantial Acquisition of Shares & Takeovers) Regulations, 2011 for Mitsubishi UFJ Financial Group (MUFG)'},
 {'date': '15 Nov,2019, 04:26PM IST',
  'text': 'Tata Global Beverages  Compliances-Reg. 39 (3) - Details of Loss of Certificate / Duplicate Certificate. Dear Sir/ Madam, \n  \n We have appended herewith the draft giving information regarding loss of share certificates. \n  \n We further request you to put the appended Notice regarding loss of share certificates for attention of the members of the Exchange, with instructions that they communicate to us immediately if they are in a position to give us information relating to any transaction or whereabouts of the original certificates.'},
 {'date': '15 Nov 2019, 10:33AM IST',
  'text': 'Share market update: FMCG shares mixed;

In [5]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

scores = []
for quote in quotes:
    score = {}
    adj_date  = quote['date'].split(',')
    yr = adj_date[1].split(' ')
    if yr[0] != '':
        score['date'] = adj_date[0] + ' ' + yr[0]
        values = analyser.polarity_scores(quote['text'])
        score['value'] = values['compound']
        scores.append(score)
scores    

[{'date': '11 Oct 2019', 'value': -0.4019},
 {'date': '11 Oct 2019', 'value': 0.296},
 {'date': '10 Oct 2019', 'value': 0.0},
 {'date': '09 Oct 2019', 'value': 0.296},
 {'date': '04 Oct 2019', 'value': 0.296},
 {'date': '04 Oct 2019', 'value': 0.7184},
 {'date': '04 Oct 2019', 'value': 0.1531},
 {'date': '03 Oct 2019', 'value': 0.296},
 {'date': '03 Oct 2019', 'value': 0.3182},
 {'date': '25 Sep 2019', 'value': 0.296},
 {'date': '23 Sep 2019', 'value': 0.0},
 {'date': '19 Sep 2019', 'value': 0.296},
 {'date': '18 Sep 2019', 'value': 0.296},
 {'date': '09 Sep 2019', 'value': -0.0},
 {'date': '05 Sep 2019', 'value': 0.296},
 {'date': '03 Sep 2019', 'value': 0.3182},
 {'date': '03 Sep 2019', 'value': 0.7003},
 {'date': '29 Aug 2019', 'value': 0.296},
 {'date': '28 Aug 2019', 'value': 0.0},
 {'date': '28 Aug 2019', 'value': 0.3182},
 {'date': '27 Aug 2019', 'value': 0.296},
 {'date': '26 Aug 2019', 'value': 0.8074},
 {'date': '22 Aug 2019', 'value': 0.296},
 {'date': '19 Aug 2019', 'value'

In [7]:
import pandas as pd
from datetime import datetime

lst = []
for scr in scores:
    tmp = {}
    strg = scr['date']
    dt = datetime.strptime(strg, '%d %b %Y')
    tmp['Date'] = dt.strftime('%Y-%m-%d')   
    tmp['Sentiment'] = scr['value']
    lst.append(tmp)
lst    

[{'Date': '2019-10-11', 'Sentiment': -0.4019},
 {'Date': '2019-10-11', 'Sentiment': 0.296},
 {'Date': '2019-10-10', 'Sentiment': 0.0},
 {'Date': '2019-10-09', 'Sentiment': 0.296},
 {'Date': '2019-10-04', 'Sentiment': 0.296},
 {'Date': '2019-10-04', 'Sentiment': 0.7184},
 {'Date': '2019-10-04', 'Sentiment': 0.1531},
 {'Date': '2019-10-03', 'Sentiment': 0.296},
 {'Date': '2019-10-03', 'Sentiment': 0.3182},
 {'Date': '2019-09-25', 'Sentiment': 0.296},
 {'Date': '2019-09-23', 'Sentiment': 0.0},
 {'Date': '2019-09-19', 'Sentiment': 0.296},
 {'Date': '2019-09-18', 'Sentiment': 0.296},
 {'Date': '2019-09-09', 'Sentiment': -0.0},
 {'Date': '2019-09-05', 'Sentiment': 0.296},
 {'Date': '2019-09-03', 'Sentiment': 0.3182},
 {'Date': '2019-09-03', 'Sentiment': 0.7003},
 {'Date': '2019-08-29', 'Sentiment': 0.296},
 {'Date': '2019-08-28', 'Sentiment': 0.0},
 {'Date': '2019-08-28', 'Sentiment': 0.3182},
 {'Date': '2019-08-27', 'Sentiment': 0.296},
 {'Date': '2019-08-26', 'Sentiment': 0.8074},
 {'Date'

In [8]:
dframe = pd.DataFrame(lst)
df = dframe.groupby('Date').mean()
df.to_csv('TATAGLOBALNEWS-1.csv', index = True)